Берёт текст, аннотирует с помощью словаря разборов, сохраняет в формате:

- 1.
- Строка предложения
- Строка со словами, поделёнными на предложения
- Строка глосс
- Перевод

In [2]:
pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import re
from razdel import sentenize
from collections import Counter, OrderedDict
import pandas as pd

In [4]:
PUNCTUATION = "!\"”“‘’#$%&'()*+, -—./:;<=>?@[\]^_`{|}~«»"

In [32]:
df = pd.read_csv('newspaper_glossing_10.csv', delimiter=';')

In [45]:
dict_w_p_g = {}
for index, row in df[:4].iterrows():
    if isinstance(row['glosses'], str):
        if isinstance(row['parts'], str):
            dict_w_p_g[row['word']] = [row['parts'], row['glosses']]

In [5]:
TRANSLIT_TABLE = {"дя" : "d'a",
    "тя" : "t'a",
    "ня" : "n'a",
    "де" : "d'e",
    "те" : "t'e",
    "че" : "če",
    "не" : "n'e",
    "фе" : "f'e",
    "дю" : "d'u",
    "тю" : "t'u",
    "ню" : "n'u",
    "дё" : "d'o",
    "тё" : "t'o",
    "нё" : "n'o",
    "сё" : "s'o",
    "ди" : "d'i",
    "а" : "a",
    "б" : "b",
    "в" : "w",
    "ҕ" : "γ",
    "г" : "g",
    "дь" : "d'",
    "ть" : "t'",
    "д" : "d",
    "ж" : "ž",
    "з" : "z",
    "и" : "i",
    "й" : "j",
    "ӄ" : "q",
    "к" : "k",
    "ль" : "l'",
    "л" : "l",
    "м" : "m",
    "нь" : "n'",
    "н" : "n",
    "ӈ" : "ŋ",
    "о" : "o",
    "п" : "p",
    "р" : "r",
    "с" : "s",
    "т" : "t",
    "у" : "u",
    "ф" : "f",
    "ч" : "č",
    "ц" : "c",
    "ш" : "š",
    "щ" : "š'",
    "ы" : "ə",
    "э" : "e",
    "ья" : "ja",
    "ью" : "ju",
    "ъе" : "je",
    "ъя" : "ja",
    "ъю" : "ju",
    "ю" : "ju",
    "я" : "ja",
    "е" : "je",
    "ъ" : "",
    "ь" : "'",
    "." : ".",
    "," : ",",
    "?" : "?",
    "!" : "!",
    "-" : "-",
    }

translit = OrderedDict(TRANSLIT_TABLE)
vowels = {"е":"\'e", "я":"\'a", "ю":"\'u", "ё":"\'o"}

In [46]:
def transliterate(string, translit, vowels):
    pattern = re.compile(r'(?<=[врпсгклзцвбмшжӈҕӄВРПСГКЛЗЦВБМШЖӇӃҔ])[еяюё]')
    string = pattern.sub(lambda x:vowels[x.group()], string)
    for cyr, ipa in translit.items():
        string = string.replace(cyr, ipa).replace(cyr.title(), ipa.title())
    return string
    
def simple_symbols(string):
    string = string.replace('γ','g').replace('q','k')
    return string

In [ ]:
filenames = []
file_list = os.listdir('data')
for filename in file_list:
    if "_eve" in filename:
        filenames.append(filename)


for filename in filenames:
    info = []
    path = 'data/'+filename
    with open(path, encoding="utf-8") as f:
        text = f.read()
    sentences = []
    for prgraph in text.split('\n'):
        for sent in list(sentenize(prgraph)):
            sentences.append(sent.text)
    sentences = [sent for sent in sentences if sent != '']  # получили предложения
    for id_sent, sent in enumerate(sentences):
        id_line = str(id_sent) + '.'
        words = sent.split()
        gloss_sent = []
        parts_sent = []
        for word in words:
            if word.strip(PUNCTUATION) != '':
                to_analyse = simple_symbols(transliterate(word.strip(PUNCTUATION).lower(),
                                       translit, vowels))
                if to_analyse in dict_w_p_g:
                    parts_sent.append(dict_w_p_g[to_analyse][0])
                    gloss_sent.append(dict_w_p_g[to_analyse][1])
                else:
                    parts_sent.append(str(to_analyse))
                    gloss_sent.append('IDK')
            else:
                words.remove(word)
        if id_sent == 0:
            id_line = id_line + ' ' + filename.strip('_eve')
        info.append([id_line, sent, ' '.join(parts_sent),
                     ' '.join(gloss_sent), 'Перевод появится'])
        new_filename = 'annotated/' + filename + '_annotated'
    with open(new_filename, 'w', encoding="utf-8") as filen:
        for infosent in info:
            for infoline in infosent:
                filen.write(infoline)
                filen.write('\n')